<a href="https://colab.research.google.com/github/akansha810/akansha810/blob/main/NLP_Bidirectional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fake News Classifier Using Bidirectional LSTM

In [ ]:
import pandas as pd


In [ ]:
df=pd.read_csv("/content/train.csv",error_bad_lines=False,engine="python")

<ipython-input-11-eeb86ccdd235>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv("/content/train.csv",error_bad_lines=False,engine="python")
Skipping line 7958: unexpected end of data


In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.isnull().values.any()

True

In [ ]:
df.isnull().sum()

id          0
title     216
author    753
text       14
label       0
dtype: int64

In [ ]:
df.dropna()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
7950,7950,Kathryn Steinle’s Family Gets Go-Ahead to Sue ...,AWR Hawkins,The parents of Kathryn Steinle have received t...,0
7951,7951,"James T. Hodgkinson, Shooting Suspect, Campaig...",Joel B. Pollak,The Washington Post reports that James T. Hodg...,0
7952,7952,"Hillary Clinton, Donald Trump, NATO: Your Thur...",Andrea Kannapell and Sandra Stevenson,(Want to get this briefing by email? Here’s th...,0
7954,7954,FBI Wiretapped Corruption Suspect Discussing C...,Daniel Greenfield,FBI Wiretapped Corruption Suspect Discussing C...,1


In [ ]:
### Get the Independent features
X=df.drop("label",axis=1)

In [ ]:
### Get the Dependent Features
y=df['label']

In [ ]:
y

0       1
1       0
2       1
3       1
4       1
       ..
7951    0
7952    0
7953    1
7954    1
7955    1
Name: label, Length: 7956, dtype: int64

In [ ]:
### Check whether dataset is balanced or not
y.value_counts()

0    3990
1    3966
Name: label, dtype: int64

In [ ]:
import tensorflow as tf

In [62]:
from tensorflow.keras.layers import Embedding,LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential


In [61]:
### Vocabulary size
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
## DataPreprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
        text = re.sub('[^a-zA-Z]',' ', messages['title'][i])
        text = text.lower()
        text = text.split()
        text = [ps.stem(word) for word in text if not word in stopwords.words("english")]
        text = " ".join(text)
        corpus.append(text)

TypeError: ignored

In [59]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'life life luxuri elton john favorit shark pictur stare long transcontinent flight',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'excerpt draft script donald trump q ampa black church pastor new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find hom

In [64]:
### One hot reprensentation
onehot_repr= [one_hot(words, voc_size)for words in corpus]
onehot_repr

[[2779, 1264, 3849, 663, 553, 391, 1864, 4395, 3939, 1305],
 [4434, 4334, 4367, 491, 3534, 3233, 2087],
 [4197, 858, 1385, 155],
 [497, 2096, 4912, 2704, 2938, 2727],
 [1968, 3534, 1898, 360, 1752, 597, 3534, 2250, 478, 1531],
 [621,
  4887,
  4238,
  3054,
  3426,
  1781,
  1821,
  3891,
  4058,
  2026,
  499,
  2478,
  3545,
  4768,
  2087],
 [3174, 3174, 2846, 1913, 2160, 2301, 1666, 4876, 818, 4822, 2718, 4219],
 [2919, 4055, 3771, 4607, 1470, 2813, 2918, 1153, 4505, 3258, 1164],
 [1798, 1614, 1240, 1328, 1781, 2104, 1776, 606, 1655, 480, 4505, 3258, 1164],
 [1496, 4554, 2761, 918, 1018, 2400, 1781, 300, 4505, 3258, 1164],
 [2868, 3105, 1500, 4959, 4920, 4210, 4824, 3376, 1781, 2388],
 [1315, 3692, 2159, 943, 2763, 777, 615, 2830],
 [1534, 1619, 4106, 4395, 2977, 2746, 508, 2970, 1161, 3238, 3694],
 [2704, 1317, 553, 4210, 1781, 1018],
 [2801, 4497, 55, 1950, 1338, 3703, 4838, 2787, 3663],
 [1732, 4683, 3769, 4264, 1430, 1399, 2098, 4505, 3258, 1164],
 [4679, 849, 790, 776, 3890, 4

In [66]:
## Padding Representation
sent_length=20
Embedded_docs=pad_sequences(onehot_repr, padding="pre", maxlen=sent_length)
Embedded_docs

array([[   0,    0,    0, ..., 4395, 3939, 1305],
       [   0,    0,    0, ..., 3534, 3233, 2087],
       [   0,    0,    0, ...,  858, 1385,  155],
       ...,
       [   0,    0,    0, ..., 1181, 4806, 1192],
       [   0,    0,    0, ..., 2814, 1018, 1660],
       [   0,    0,    0, ...,  545, 1814, 2087]], dtype=int32)

In [70]:
from unicodedata import bidirectional
# Creating Model
embedding_vector_features=40
model=Sequential()
#Embedding
model.add(Embedding(voc_size, embedding_vector_features,input_length=sent_length))
##model.add(LSTM(100))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [75]:
import numpy as np
X_final= np.array(Embedded_docs)
y_final=np.array(y)

In [77]:
X_final.shape

(53, 20)

In [78]:
### Train-Test_split
from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test= train_test_split(X_final, y_final, test_size=0.33,random_state=42)

ValueError: ignored

In [73]:
### Model Training
model.fit(X_train , y_train, validation_data=(X_test,y_test),epochs=10,batch_size=32)

NameError: ignored

In [80]:
y_pred = model.predict(X_test)

NameError: ignored

In [79]:
import numpy as np
y_pred=np.where(y_pred>=0.5,1,0)

NameError: ignored

In [81]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
confusion_matrix(y_pred,y_test)

NameError: ignored

In [ ]:
accuracy_score(y_pred,y_test)

In [ ]:
classification_report(y_pred,y_test)